### Notebook to visualise the dynamics learned using LKmeans and LCTA on the robustness test set


In [1]:
using CairoMakie
using XLSX
using DataFrames
using Statistics

In [2]:
file_path = "C:/Users/20192809/OneDrive - TU Eindhoven/TUe/Thesis/Thesis writing/Combined results.xlsx"
sheet = "Robustness_LCTA"

df = DataFrame(XLSX.readtable(file_path, sheet, "F:H", first_row = 2))
stds = [0.1, 0.35]
timepoints = [0,1,2,3,4]
initial_k5 = 0.04
initial_k6 = 3.0
true_k5_linear_rate = -0.1 * initial_k5
true_k6_linear_rate = 0.25 * initial_k6
true_k6_quadratic_rate = -0.08 * initial_k6;

In [3]:
# extract LKmeans progressions 
df_LKmeans = DataFrame(XLSX.readtable(file_path, "Robustness_LKM_SR", "F:G", first_row = 17))

# Hardcoded from combined results
# Noise Level 0.1
k5_01_rate = -0.00394
k5_01_const = 0.0399
k6_01_quad = -0.242
k6_01_lin = 0.760
k6_01_const = 2.98

# Noise Level 0.35
k5_035_rate = -0.00326
k5_035_const = 0.0389
k6_035_quad = -0.279
k6_035_lin = 0.911
k6_035_const = 2.84



2.84

In [4]:
# Function to extract and average values from each cell
function extract_average(val)
    # Replace ',' with '.' for decimal consistency
    val = replace(string(val), ',' => '.')
    nums = parse.(Float64, split(val, " & "))
    return mean(nums)
end

# Apply to all columns
k5_linear = extract_average.(df[1:5, 1])
k6_linear = extract_average.(df[1:5, 2])
k6_quadratic = extract_average.(df[1:5, 3]);


In [5]:
# Compute true k5 (linear decrease)
true_k5 = initial_k5 .+ true_k5_linear_rate .* timepoints

# Compute true k6 (quadratic progression)
true_k6 = initial_k6 .+ true_k6_linear_rate .* timepoints .+ true_k6_quadratic_rate .* (timepoints .^ 2);
# Compute predicted k5 and k6 from LCTA
predicted_k5 = [initial_k5 .+ k5_linear[i] .* timepoints for i in 1:length(stds)]
predicted_k6 = [initial_k6 .+ k6_linear[i] .* timepoints .+ true_k6_quadratic_rate .* (timepoints .^ 2) for i in 1:length(stds)]


2-element Vector{Vector{Float64}}:
 [3.0, 3.5108499999999996, 3.5416999999999996, 3.09255, 2.1634]
 [3.0, 3.513, 3.5460000000000003, 3.099, 2.1720000000000006]

In [7]:
using CairoMakie

# Create figure
fig = Figure(resolution=(800, 400))

# New timepoints for smoothing (finer resolution)
fine_timepoints = 0:0.1:4  # For example, using 0.1 intervals

# Fine initial values for k5 and k6 over the new timepoints
fine_initial_k5 = initial_k5 .+ 0 .* fine_timepoints  # Just repeat initial_k5 over all timepoints
fine_initial_k6 = initial_k6 .+ 0 .* fine_timepoints  # Similarly for k6

# Extract averages for the k5 and k6 rates for selected noise levels (0.1 and 0.35)
selected_stds = [3, 5]  # Indices corresponding to 0.1 and 0.35 in df
k5_linear = extract_average.(df[selected_stds, 1])
k6_linear = extract_average.(df[selected_stds, 2])
k6_quadratic = extract_average.(df[selected_stds, 3])

# True dynamics for k5 and k6 over fine timepoints
true_k5 = fine_initial_k5 .+ true_k5_linear_rate .* fine_timepoints
true_k6 = fine_initial_k6 .+ true_k6_linear_rate .* fine_timepoints .+ true_k6_quadratic_rate .* (fine_timepoints .^ 2)

# Set up axes with specified label settings
ax1 = Axis(fig[1, 1], xlabel="τ (years)", ylabel="k5", 
           xlabelsize=20, ylabelsize=16, 
           xticklabelsize=14, yticklabelsize=14, 
           xlabelfont=:bold, ylabelfont=:bold)

ax2 = Axis(fig[1, 2], xlabel="τ (years)", ylabel="k6", 
           xlabelsize=20, ylabelsize=16, 
           xticklabelsize=14, yticklabelsize=14, 
           xlabelfont=:bold, ylabelfont=:bold)

# Plot true dynamics with a thick black line (linewidth = 5)
lines!(ax1, fine_timepoints, true_k5, color=:black,  linewidth=5, alpha=1)
lines!(ax2, fine_timepoints, true_k6, color=:black, linewidth=5, alpha=1)

# Use colormap for predicted dynamics
colormap = Makie.wong_colors()

# Plot LCTA predicted dynamics
for (i, idx) in enumerate(selected_stds)
    predicted_k5_fine = fine_initial_k5 .+ k5_linear[i] .* fine_timepoints
    predicted_k6_fine = fine_initial_k6 .+ k6_linear[i] .* fine_timepoints .+ k6_quadratic[i] .* (fine_timepoints .^ 2)

    # Plot LCTA with solid lines
    lines!(ax1, fine_timepoints, predicted_k5_fine, color=colormap[i], linewidth=5, alpha=0.8)
    lines!(ax2, fine_timepoints, predicted_k6_fine, color=colormap[i], linewidth=5, alpha=0.8)
end

# SR dynamics (dashed lines)
sr_k5_01 = fine_initial_k5 .+ k5_01_rate .* fine_timepoints
sr_k6_01 = fine_initial_k6 .+ k6_01_lin .* fine_timepoints .+ k6_01_quad .* (fine_timepoints .^ 2)

sr_k5_035 = fine_initial_k5 .+ k5_035_rate .* fine_timepoints
sr_k6_035 = fine_initial_k6 .+ k6_035_lin .* fine_timepoints .+ k6_035_quad .* (fine_timepoints .^ 2)

# Plot SR dynamics using dashed lines
lines!(ax1, fine_timepoints, sr_k5_01, color=colormap[1], linestyle=:dash, linewidth=5, alpha=0.8)
lines!(ax2, fine_timepoints, sr_k6_01, color=colormap[1], linestyle=:dash, linewidth=5, alpha=0.5)

lines!(ax1, fine_timepoints, sr_k5_035, color=colormap[2], linestyle=:dash, linewidth=5, alpha=0.8)
lines!(ax2, fine_timepoints, sr_k6_035, color=colormap[2], linestyle=:dash, linewidth=5, alpha=0.5)

# Create a custom legend
legend_elements = [
    LineElement(color=:black, linestyle=:dash, linewidth=5),  # True k5 and k6
    LineElement(color=colormap[1], linewidth=5),  # LCTA (0.1)
    LineElement(color=colormap[2], linewidth=5),  # LCTA (0.35)
    LineElement(color=colormap[1], linestyle=:dash, linewidth=2),  # SR (0.1)
    LineElement(color=colormap[2], linestyle=:dash, linewidth=2)   # SR (0.35)
]

legend_labels = [
    "True Dynamics",
    "LCTA (noise level 0.2)",
    "LCTA (noise level 0.35)",
    "SR (noise level 0.2)",
    "SR (noise level 0.35)"
]

# Add legends
Legend(fig[1, 3], legend_elements, legend_labels, position=:right)

# Show plot
fig

# Save the plot
save("LCTA_SR_dynamics.png", fig)


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\20192809\.julia\packages\Makie\VRavR\src\scenes.jl:220


CairoMakie.Screen{IMAGE}
